In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import namedtuple
from datetime import date
from datetime import date, datetime, timedelta
from logging import INFO, basicConfig, getLogger
from sys import stdout
from typing import Dict, Generator, Tuple, Sequence, Optional

import numpy as np
import pandas as pd
import geopandas as gpd

Disposition = namedtuple("Disposition", ("rate", "days"))


class Parameters:
    def __init__(self, **kwargs):
        self.population=3600000
        self.current_hospitalized=69
        self.date_first_hospitalized=None
        self.doubling_time=4.0
        self.hospitalized=Disposition(0.025, 7)
        self.icu=Disposition(0.0075, 9)
        self.infectious_days=14
        self.market_share=0.15
        self.n_days=100
        self.current_date=date.today()
        self.mitigation_date = None
        self.relative_contact_rate=0.3
        self.ventilated=Disposition(0.005, 10)
        self.recovered = 0
        
        if bool(kwargs):
            self.population = kwargs.get("population",self.population)
            self.current_hospitalized = kwargs.get("current_hospitalized",self.current_hospitalized)
            self.hospitalized=Disposition(kwargs.get("hospitalized")/100, 7)
            self.relative_contact_rate=kwargs.get("relative_contact_rate")/100
        
        self.dispositions = {
            "hospitalized": self.hospitalized,
            "icu": self.icu,
            "ventilated": self.ventilated,
        }


basicConfig(
    level=INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    stream=stdout,
)
logger = getLogger(__name__)


class SimSirModel:

    def __init__(self, p: Parameters):

        self.rates = {
            key: d.rate
            for key, d in p.dispositions.items()
        }

        self.days = {
            key: d.days
            for key, d in p.dispositions.items()
        }

        self.keys = ("susceptible", "infected", "recovered")

        # An estimate of the number of infected people on the day that
        # the first hospitalized case is seen
        #
        # Note: this should not be an integer.
        infected = (
            1.0 / p.market_share / p.hospitalized.rate
        )

        susceptible = p.population - infected

        gamma = 1.0 / p.infectious_days
        self.gamma = gamma

        self.susceptible = susceptible
        self.infected = infected
        self.recovered = p.recovered

        if p.date_first_hospitalized is None and p.doubling_time is not None:
            # Back-projecting to when the first hospitalized case would have been admitted
            logger.info('Using doubling_time: %s', p.doubling_time)

            intrinsic_growth_rate = get_growth_rate(p.doubling_time)

            self.beta = get_beta(intrinsic_growth_rate,  gamma, self.susceptible, 0.0)
            self.beta_t = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, p.relative_contact_rate)

            self.i_day = 0 # seed to the full length
            self.run_projection(p, [(self.beta, p.n_days)])
            self.i_day = i_day = int(get_argmin_ds(self.census_df, p.current_hospitalized))

            self.run_projection(p, self.gen_policy(p))

            logger.info('Set i_day = %s', i_day)
            p.date_first_hospitalized = p.current_date - timedelta(days=i_day)
            logger.info(
                'Estimated date_first_hospitalized: %s; current_date: %s; i_day: %s',
                p.date_first_hospitalized,
                p.current_date,
                self.i_day)

        elif p.date_first_hospitalized is not None and p.doubling_time is None:
            # Fitting spread parameter to observed hospital census (dates of 1 patient and today)
            self.i_day = (p.current_date - p.date_first_hospitalized).days
            self.current_hospitalized = p.current_hospitalized
            logger.info(
                'Using date_first_hospitalized: %s; current_date: %s; i_day: %s, current_hospitalized: %s',
                p.date_first_hospitalized,
                p.current_date,
                self.i_day,
                p.current_hospitalized,
            )

            # Make an initial coarse estimate
            dts = np.linspace(1, 15, 15)
            min_loss = self.get_argmin_doubling_time(p, dts)

            # Refine the coarse estimate
            for iteration in range(4):
                dts = np.linspace(dts[min_loss-1], dts[min_loss+1], 15)
                min_loss = self.get_argmin_doubling_time(p, dts)

            p.doubling_time = dts[min_loss]

            logger.info('Estimated doubling_time: %s', p.doubling_time)
            intrinsic_growth_rate = get_growth_rate(p.doubling_time)
            self.beta = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, 0.0)
            self.beta_t = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, p.relative_contact_rate)
            self.run_projection(p, self.gen_policy(p))

            self.population = p.population
        else:
            logger.info(
                'doubling_time: %s; date_first_hospitalized: %s',
                p.doubling_time,
                p.date_first_hospitalized,
            )
            raise AssertionError('doubling_time or date_first_hospitalized must be provided.')

        logger.info('len(np.arange(-i_day, n_days+1)): %s', len(np.arange(-self.i_day, p.n_days+1)))
        logger.info('len(raw_df): %s', len(self.raw_df))

        self.infected = self.raw_df['infected'].values[self.i_day]
        self.susceptible = self.raw_df['susceptible'].values[self.i_day]
        self.recovered = self.raw_df['recovered'].values[self.i_day]

        self.intrinsic_growth_rate = intrinsic_growth_rate

        # r_t is r_0 after distancing
        self.r_t = self.beta_t / gamma * susceptible
        self.r_naught = self.beta / gamma * susceptible

        doubling_time_t = 1.0 / np.log2(
            self.beta_t * susceptible - gamma + 1)
        self.doubling_time_t = doubling_time_t
        
        #SIM_SIR_DF - 3rd Plot
        self.sim_sir_w_date_df = build_sim_sir_w_date_df(self.raw_df, p.current_date, self.keys)

        self.sim_sir_w_date_floor_df = build_floor_df(self.sim_sir_w_date_df, self.keys)
        self.admits_floor_df = build_floor_df(self.admits_df, p.dispositions.keys())
        self.census_floor_df = build_floor_df(self.census_df, p.dispositions.keys())

        self.daily_growth_rate = get_growth_rate(p.doubling_time)
        self.daily_growth_rate_t = get_growth_rate(self.doubling_time_t)

    def get_argmin_doubling_time(self, p: Parameters, dts):
        losses = np.full(dts.shape[0], np.inf)
        for i, i_dt in enumerate(dts):
            intrinsic_growth_rate = get_growth_rate(i_dt)
            self.beta = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, 0.0)
            self.beta_t = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, p.relative_contact_rate)

            self.run_projection(p, self.gen_policy(p))

            # Skip values the would put the fit past peak
            peak_admits_day = self.admits_df.hospitalized.argmax()
            if peak_admits_day < 0:
                continue

            loss = self.get_loss()
            losses[i] = loss

        min_loss = pd.Series(losses).argmin()
        return min_loss

    def gen_policy(self, p: Parameters) -> Sequence[Tuple[float, int]]:
        if p.mitigation_date is not None:
            mitigation_day = -(p.current_date - p.mitigation_date).days
        else:
            mitigation_day = 0

        total_days = self.i_day + p.n_days

        if mitigation_day < -self.i_day:
            mitigation_day = -self.i_day

        pre_mitigation_days = self.i_day + mitigation_day
        post_mitigation_days = total_days - pre_mitigation_days

        return [
            (self.beta,   pre_mitigation_days),
            (self.beta_t, post_mitigation_days),
        ]

    def run_projection(self, p: Parameters, policy: Sequence[Tuple[float, int]]):
        self.raw_df = sim_sir_df(
            self.susceptible,
            self.infected,
            p.recovered,
            self.gamma,
            -self.i_day,
            policy
        )

        self.dispositions_df = build_dispositions_df(self.raw_df, self.rates, p.market_share, p.current_date)
        #Projected Admissions plot - Plot 1
        self.admits_df = build_admits_df(self.dispositions_df)
        #Projected Census plot - Plot 2
        self.census_df = build_census_df(self.admits_df, self.days)
        self.current_infected = self.raw_df.infected.loc[self.i_day]

    def get_loss(self) -> float:
        """Squared error: predicted vs. actual current hospitalized."""
        predicted = self.census_df.hospitalized.loc[self.i_day]
        return (self.current_hospitalized - predicted) ** 2.0


def get_argmin_ds(census_df: pd.DataFrame, current_hospitalized: float) -> float:
    # By design, this forbids choosing a day after the peak
    # If that's a problem, see #381
    peak_day = census_df.hospitalized.argmax()
    losses_df = (census_df.hospitalized[:peak_day] - current_hospitalized) ** 2.0
    return losses_df.argmin()


def get_beta(
    intrinsic_growth_rate: float,
    gamma: float,
    susceptible: float,
    relative_contact_rate: float
) -> float:
    return (
        (intrinsic_growth_rate + gamma)
        / susceptible
        * (1.0 - relative_contact_rate)
    )


def get_growth_rate(doubling_time: Optional[float]) -> float:
    """Calculates average daily growth rate from doubling time."""
    if doubling_time is None or doubling_time == 0.0:
        return 0.0
    return (2.0 ** (1.0 / doubling_time) - 1.0)


def sir(
    s: float, i: float, r: float, beta: float, gamma: float, n: float
) -> Tuple[float, float, float]:
    """The SIR model, one time step."""
    s_n = (-beta * s * i) + s
    i_n = (beta * s * i - gamma * i) + i
    r_n = gamma * i + r

    # TODO:
    #   Post check dfs for negative values and
    #   warn the user that their input data is bad.
    #   JL: I suspect that these adjustments covered bugs.

    #if s_n < 0.0:
    #    s_n = 0.0
    #if i_n < 0.0:
    #    i_n = 0.0
    #if r_n < 0.0:
    #    r_n = 0.0
    scale = n / (s_n + i_n + r_n)
    return s_n * scale, i_n * scale, r_n * scale


def gen_sir(
    s: float, i: float, r: float, gamma: float, i_day: int, policies: Sequence[Tuple[float, int]]
) -> Generator[Tuple[int, float, float, float], None, None]:
    """Simulate SIR model forward in time yielding tuples.
    Parameter order has changed to allow multiple (beta, n_days)
    to reflect multiple changing social distancing policies.
    """
    s, i, r = (float(v) for v in (s, i, r))
    n = s + i + r
    d = i_day
    for beta, n_days in policies:
        for _ in range(n_days):
            yield d, s, i, r
            s, i, r = sir(s, i, r, beta, gamma, n)
            d += 1
    yield d, s, i, r


def sim_sir_df(
    s: float, i: float, r: float,
    gamma: float, i_day: int, policies: Sequence[Tuple[float, int]]
) -> pd.DataFrame:
    """Simulate the SIR model forward in time."""
    return pd.DataFrame(
        data=gen_sir(s, i, r, gamma, i_day, policies),
        columns=("day", "susceptible", "infected", "recovered"),
    )


def build_sim_sir_w_date_df(
    raw_df: pd.DataFrame,
    current_date: datetime,
    keys: Sequence[str],
) -> pd.DataFrame:
    day = raw_df.day
    return pd.DataFrame({
        "day": day,
        "date": day.astype('timedelta64[D]') + np.datetime64(current_date),
        **{
            key: raw_df[key]
            for key in keys
        }
    })


def build_floor_df(df, keys):
    """Build floor sim sir w date."""
    return pd.DataFrame({
        "day": df.day,
        "date": df.date,
        **{
            key: np.floor(df[key])
            for key in keys
        }
    })


def build_dispositions_df(
    raw_df: pd.DataFrame,
    rates: Dict[str, float],
    market_share: float,
    current_date: datetime,
) -> pd.DataFrame:
    """Build dispositions dataframe of patients adjusted by rate and market_share."""
    patients = raw_df.infected + raw_df.recovered
    day = raw_df.day
    return pd.DataFrame({
        "day": day,
        "date": day.astype('timedelta64[D]') + np.datetime64(current_date),
        **{
            key: patients * rate * market_share
            for key, rate in rates.items()
        }
    })


def build_admits_df(dispositions_df: pd.DataFrame) -> pd.DataFrame:
    """Build admits dataframe from dispositions."""
    admits_df = dispositions_df - dispositions_df.shift(1)
    admits_df.day = dispositions_df.day
    admits_df.date = dispositions_df.date
    return admits_df


def build_census_df(
    admits_df: pd.DataFrame,
    lengths_of_stay: Dict[str, int],
) -> pd.DataFrame:
    """Average Length of Stay for each disposition of COVID-19 case (total guesses)"""
    return pd.DataFrame({
        'day': admits_df.day,
        'date': admits_df.date,
        **{
            key: (
                admits_df[key].cumsum()
                - admits_df[key].cumsum().shift(los).fillna(0)
            )
            for key, los in lengths_of_stay.items()
        }
    })


#Convert the current dataframe to stacked groupby format
def pivot(df, state=None):
    df_copy = df.copy()
    if state:
        return (df_copy.set_index(["day", "date", "date_str", "state", "countyname"]).stack().reset_index(name="Value").rename(columns={'level_5': "status"}))
    return (df_copy.set_index(["day", "date", "date_str", "countyname"]).stack().reset_index(name="Value").rename(columns={'level_4': "status"}))



def run_geo_merge_df(df, geodf):
    return geodf.merge(df, on="countyname")


#Adds county, date_str (date as string), state (if exists) and calls the pivot definition
def modify_df(df, county, state=None):
    if state:
        df['state'] = state
    df['countyname'] = county
    df['date_str'] = df['date'].astype('str')
    
    return pivot(df,state)



# Runs the model for the county with or without geometries
def run_model_for_county(df, county):
    df.columns = map(str.lower, df.columns)
    if all(col in df.columns.values for col in ['countyname', 'hospitaliz', 'population', 'unacast_ch', 'hospital_1']):
        if df.loc[df['countyname'] == county].shape[0]:
            c= df.loc[df['countyname'] == county].iloc[0]
            p = Parameters(population=c.population,
                           current_hospitalized=c.hospitaliz,
                           relative_contact_rate=c.unacast_ch,
                           hospitalized=c.hospital_1)
            m = SimSirModel(p)
            pivot_admits = modify_df(m.admits_floor_df, county)
            pivot_census = modify_df(m.census_floor_df, county)
            pivot_sir = modify_df(m.sim_sir_w_date_floor_df, county)
            return pivot_admits, pivot_census, pivot_sir
        else:
            print("County {} doesn't exist".format(county))
    
    else:
        print('Required column names do not exists')



# Runs the model for the state/county with or without geometries
def run_model_for_state_county(df, state, county):
    df.columns = map(str.lower, df.columns)
    if all(col in df.columns.values for col in ['statename','countyname', 'hospitaliz', 'population', 'unacast_ch', 'hospital_1']):
        if df.loc[(df['statename'] == state) & (df['countyname'] == county)].shape[0]:
            c= df.loc[(df['statename'] == state) & (df['countyname'] == county)].iloc[0]
            p = Parameters(population=c.population,
                           current_hospitalized=c.hospitaliz,
                           relative_contact_rate=c.unacast_ch,
                           hospitalized=c.hospital_1)
            m = SimSirModel(p)
            pivot_admits = modify_df(m.admits_floor_df, county, state)
            pivot_census = modify_df(m.census_floor_df, county, state)
            pivot_sir = modify_df(m.sim_sir_w_date_floor_df, county, state)
            return pivot_admits, pivot_census, pivot_sir
        else:
            print("County {} doesn't exist".format(county))
    
    else:
        print('Required column names do not exists')


def merge_dfs(admits_df, census_df, sir_df):
    admits_df_cpy = admits_df.copy()
    census_df_cpy = census_df.copy()
    sir_df_cpy = sir_df.copy()
    admits_df_cpy['type'] = 'admits'
    census_df_cpy['type'] = 'census'
    sir_df_cpy['type'] = 'sim_sir'
    return pd.concat([admits_df_cpy, census_df_cpy, sir_df_cpy], ignore_index=True)

#This runs the model for all the counties if you have the county name in the dataset
#(Column name should be 'countyname')
def run_model_for_all_counties(df):
    df.columns = map(str.lower, df.columns)
    all_df = pd.DataFrame()
    for index, row in df.iterrows():
        admits_df, census_df, sir_df = run_model_for_county(df,(row['countyname']))
        fin_df = merge_dfs(admits_df, census_df, sir_df)
        all_df = all_df.append(fin_df)
    return all_df


#This runs the model for all the states if you have the statename and respective county name in the dataset
#(Column names should be 'statename' and 'countyname')
def run_model_for_all_states(df):
    df.columns = map(str.lower, df.columns)
    all_df = pd.DataFrame()
    for index, row in df.iterrows():
        admits_df, census_df, sir_df = run_model_for_state_county(df,(row['statename']),(row['countyname']))
        fin_df = merge_dfs(admits_df, census_df, sir_df)
        all_df = all_df.append(fin_df)
    return all_df


In [ ]:
#DND the 5 fields(Countyname, Hospitalz, Unacast_ch, Hospital_1, Population) from Chime DS
chime_df = #pd.read_csv('CHIMEDataSet.csv')
county_run_df = run_model_for_all_counties(chime_df)

In [ ]:
%insights_return(county_run_df)

In [ ]:
#DND the 6 fields(Statename, Countyname, Hospitalz, Unacast_ch, Hospital_1, Population) from USA State county DS
#usa_state_county_df = #pd.read_csv('USStatesCHIME.csv')
#state_run_df = run_model_for_all_states(usa_state_county_df)

In [ ]:
#%insights_return(state_run_df)